# 加载 Himawari-8 AHI 数据并生成真彩色RGB

## 加载数据

In [1]:
from satpy import Scene, find_files_and_readers
from satpy.resample import get_area_def
from datetime import datetime

<p>我们首先定位并指定要读取的AHI数据：</p>

In [2]:
files = find_files_and_readers(start_time=datetime(2015, 2, 7, 3, 0),
                               end_time=datetime(2015, 2, 7, 3, 10),
                               base_dir="/data/satellite/Himawari-8/HSD",
                               reader='ahi_hsd')

此示例正在加载以Himawari标准数据（HSD）格式存储的Himawari数据。但您也可以使用相同的代码加载HRIT格式的数据，只需在上述参数列表中指定适当的读取器（`ahi_hrit`）即可。
<p>我们创建场景对象</p>

In [3]:
scn = Scene(sensor='ahi', filenames=files)

此处我们加载 `真彩色` 合成

In [4]:
rgbname = 'true_color'

In [5]:
scn.load([rgbname])

/home/a001673/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 生成完整磁盘镜像
请注意，此步骤可能需要处理设备上可用超过10 GB的内存（取决于CPU核心数量）

In [6]:
new_scn = scn.resample(scn.min_area(), resampler='native')

上述语句将分辨率降低到较粗的通道。如果你想生成全分辨率图像，请改用重采样调用中的 `scn.max_area()`。这将需要相当多的资源。

In [7]:
new_scn.save_dataset(rgbname, filename='himawari_ahi_truecolor_{datetime}.png'.format(datetime=scn.start_time.strftime('%Y%m%d%H%M')))


<img src='https://media.githubusercontent.com/media/pytroll/pytroll-examples/321c1cc5799832b8cc5b9e0994a0c461cf11213d/satpy/himawari_ahi_truecolor_201502070300_thumb.png'>

## 在感兴趣区域上生成复合图像现在让我们定义一个地图投影和一个子区域，并将数据投影到该区域上。我们使用 Pyresample 来定义区域。此定义也可以放入 area.yaml 配置文件中。

In [8]:
from pyresample.utils import get_area_def
area_id = 'japan'
x_size = 2407
y_size = 1655
area_extent = (-1203877.326193321, -754860.2505908236, 1203877.3261933187, 900373.6358697552)
projection = '+proj=laea +lat_0=37.5 +lon_0=137.5 +ellps=WGS84'
description = "Japan"
proj_id = 'laea_137.5_37.5'
areadef = get_area_def(area_id, description, proj_id, projection,x_size, y_size, area_extent)

In [9]:
local_scene = scn.resample(areadef)

In [10]:
local_scene.show(rgbname)

/home/a001673/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/home/a001673/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in log10
  return func(*args2)


<img src='https://media.githubusercontent.com/media/pytroll/pytroll-examples/321c1cc5799832b8cc5b9e0994a0c461cf11213d/satpy/himawari_ahi_truecolor_201502070300_japan_thumb.png'>